In [1]:
import pandas as pd
import numpy as np
import math
import statistics
from statistics import median

In [2]:
project_list = ['heroku.csv', 'rails.csv', 'gradle.csv', 'jruby.csv', 'metasploit-framework.csv', 'cloudify.csv', 'vagrant.csv', 'rubinius.csv', 'open-build-service.csv', 'sonarqube.csv', 'loomio.csv', 'fog.csv', 'opal.csv', 'cloud_controller_ng.csv', 'puppet.csv', 'concerto.csv', 'sufia.csv', 'geoserver.csv', 'orbeon-forms.csv', 'graylog2-server.csv']

In [3]:
print(len(project_list))

20


In [4]:
algorithms = ['BATCH4', 'BATCHBISECT', 'BATCHSTOP4']
batch_sizes = [2, 4, 8, 16]

In [5]:
result_rows = {}

In [6]:
def get_results(results):
    l = []
    
    for r in results:
        if r == 'passed':
            l.append(1)
        else:
            l.append(0)
            
    return l

In [7]:
def str_to_list(s):
    if s == '[]':
        return [0]
    l = s[1:-1].split(', ')
    l = [int(x) for x in l]
    return l

In [8]:
def get_delay_from_ci(ci, y_test, batch_size):
    
    i = 0        
    delay_list = []
    missed = []
    batch_list = []
    sbs_list = []
    delay_list = []
    
    b = batch_size-1

    while i < len(ci):
        if ci[i] == 0:

            while len(missed) > 0:
                ind = missed.pop()
                sbs_list.append(i - ind)
                print('{} {} {}'.format(ind, i, i-ind))

            batch_list.append(b)
            b -= 1
            if b == -1:
                b = batch_size - 1
            
            #print(y_test[i], ci[i], sbs_delay, batch_delay, sbs_delay + batch_delay)

        if ci[i] == 1:
            if y_test[i] == 0:
                missed.append(i)

        i += 1

    while len(missed) > 0:
            ind = missed.pop()
            sbs_list.append(i - ind)
            print('{} {} {}'.format(ind, i, i-ind))
    
    delay_list.extend(sbs_list)
    delay_list.extend(batch_list)
    
    return delay_list

In [9]:
for p in project_list:
    
    result_rows[p] = []
    filename = p + '_sw.csv'
    p_data = pd.read_csv(filename)
    if 'test_start' in p_data.columns:
        print('renaming columns')
        p_data = p_data.rename(columns={'test_start':'start_p', 'test_end':'end_p'})
    
    test_file = 'project_metrics/' + p.split('.')[0] + '_metrics.csv'
    test_data = pd.read_csv(test_file)
    
    window_size = len(test_data)//5
    
    for alg in algorithms:
        
        for b in batch_sizes:
            
            if alg == 'BATCH4':
                if b != 4:
                    continue
            elif alg == 'BATCHSTOP4':
                if b < 4:
                    continue
            
            b_data = p_data[ (p_data['algorithm'] == alg) & (p_data['batch_size'] == b)]
            b_data = b_data.sort_values(by="start_p")
            
            cur_row = [p, alg, b, 0, 0, [], 0, 0, 0]
            
            for i in range(len(b_data)):
                row = b_data.iloc[i]
                print(row)
                
                
                test_start = row['start_p']
                test_end = row['end_p']
                test_size = test_end - test_start
                
                #print(len(test_data), window_size, test_start, test_end, test_size, row['testall_size'])
                if test_size != row['testall_size']:
                    print('PROBLEM!! Window not matching')
                
                y_test = get_results(test_data.iloc[test_start:test_end]['tr_status'].tolist())
                row_ci = str_to_list(row['ci'])
                print(row_ci, y_test)
                delay = get_delay_from_ci(row_ci, y_test, b)
                
                cur_row[4] = ((cur_row[4]*cur_row[3]) + (row['builds_reqd']*row['testall_size']))/(cur_row[3]+row['testall_size'])
                cur_row[3] += row['testall_size']
                cur_row[5].extend(delay)
                cur_row[6] += row['batch_duration']
                cur_row[7] += row['actual_duration']
                cur_row[8] = median(cur_row[5])
            
            result_rows[p].append(cur_row)
        
        print(alg + ' completed')
    
    print(p + ' completed')

                
            
            
    

renaming columns
Unnamed: 0                                                         0
project                                                   heroku.csv
start_p                                                          291
end_p                                                            416
algorithm                                                     BATCH4
batch_size                                                         4
builds_reqd                                                     54.4
median_delay                                                     1.5
delay_list         [1, 5, 3, 2, 1, 0, 3, 2, 1, 0, 3, 2, 1, 0, 3, ...
actual_duration                                                41803
batch_duration                                                 24855
testall_size                                                     125
ci                 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, ...
Name: 0, dtype: object
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0,

278 279 1
277 279 2
276 279 3
330 331 1
329 331 2
328 331 3
327 331 4
326 331 5
325 331 6
323 331 8
321 331 10
320 331 11
319 331 12
318 331 13
362 365 3
359 365 6
358 365 7
357 365 8
353 365 12
349 365 16
344 365 21
477 479 2
476 479 3
473 479 6
472 479 7
471 479 8
470 479 9
469 479 10
468 479 11
467 479 12
463 479 16
435 479 44
434 479 45
426 479 53
401 479 78
400 479 79
398 479 81
395 479 84
391 479 88
390 479 89
383 479 96
382 479 97
381 479 98
380 479 99
375 479 104
371 479 108
369 479 110
494 500 6
493 500 7
492 500 8
483 500 17
571 585 14
563 585 22
562 585 23
561 585 24
560 585 25
559 585 26
558 585 27
557 585 28
655 670 15
654 670 16
653 670 17
650 670 20
646 670 24
645 670 25
644 670 26
643 670 27
642 670 28
641 670 29
640 670 30
639 670 31
638 670 32
637 670 33
635 670 35
701 702 1
700 702 2
679 702 23
677 702 25
676 702 26
675 702 27
728 736 8
742 749 7
741 749 8
773 775 2
841 846 5
831 846 15
827 846 19
826 846 20
Unnamed: 0                                                 

[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

93 126 33
92 126 34
91 126 35
90 126 36
89 126 37
88 126 38
87 126 39
86 126 40
85 126 41
76 126 50
75 126 51
74 126 52
73 126 53
72 126 54
71 126 55
70 126 56
69 126 57
63 126 63
62 126 64
61 126 65
60 126 66
59 126 67
58 126 68
57 126 69
56 126 70
55 126 71
54 126 72
53 126 73
52 126 74
51 126 75
50 126 76
49 126 77
43 126 83
42 126 84
41 126 85
40 126 86
39 126 87
31 126 95
30 126 96
27 126 99
26 126 100
25 126 101
24 126 102
177 181 4
176 181 5
175 181 6
174 181 7
173 181 8
172 181 9
171 181 10
170 181 11
169 181 12
168 181 13
167 181 14
166 181 15
165 181 16
164 181 17
163 181 18
162 181 19
161 181 20
160 181 21
159 181 22
153 181 28
152 181 29
225 229 4
224 229 5
223 229 6
222 229 7
221 229 8
219 229 10
208 229 21
204 229 25
201 229 28
200 229 29
197 229 32
195 229 34
194 229 35
193 229 36
190 229 39
189 229 40
188 229 41
187 229 42
186 229 43
264 265 1
263 265 2
262 265 3
261 265 4
260 265 5
257 265 8
308 309 1
306 309 3
305 309 4
304 309 5
303 309 6
302 309 7
384 385 1
369 385 

...

Name: 62, dtype: object
[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0] [0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1,

74 77 3
73 77 4
72 77 5
71 77 6
70 77 7
69 77 8
68 77 9
67 77 10
64 77 13
Unnamed: 0                                                        15
project                                                sonarqube.csv
start_p                                                          758
end_p                                                            948
algorithm                                                BATCHBISECT
batch_size                                                         4
builds_reqd                                                74.736842
median_delay                                                     2.0
delay_list         [3, 2, 1, 0, 3, 2, 1, 0, 3, 2, 1, 0, 3, 2, 1, ...
actual_duration                                               454064
batch_duration                                                348523
testall_size                                                     190
ci                 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: 15, dtype: object
[0, 0

Name: 34, dtype: object
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] [1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [10]:
frames = []

In [11]:
for key in result_rows:
    proj_res = result_rows[key]
    df = pd.DataFrame(proj_res, columns=['project', 'algorithm', 'batch_size', 'testall_size', 'builds_reqd', 'delay_list', 'build_duration', 'actual_duration', 'median_delay'])
    frames.append(df)
    

In [12]:
result = pd.concat(frames)

In [13]:
result.to_csv("final_mlci_results.csv")

In [38]:
848*9

7632